In [50]:
import numpy as np
import matplotlib.pyplot as plt

import skimage
from PIL import Image
from skimage import transform, morphology, feature, filters
from scipy.ndimage.interpolation import shift
from tqdm import tqdm_notebook
from sys import getsizeof
from sklearn import neighbors, model_selection

In [2]:
plt.rcParams['figure.figsize'] = (10, 15)

In [3]:
from utils.photo_processing import ImageProcessor

In [4]:
im = Image.open("data/originals/PRISM1.jpg")
im = im.convert('L')
im = np.asarray(im)
im = transform.resize(im, (150, 150))

/home/bartek/.pyenv/versions/3.6.1/envs/woarld/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/bartek/.pyenv/versions/3.6.1/envs/woarld/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [5]:
np.mean(im)

0.7315446574679253

### Generate samples

In [6]:
angles = np.arange(-15, 15 + 1, 7.5)
shift_x = np.arange(-30, 30 + 1, 15)
shift_y = shift_x
fliplr = [True, False]
rescale_x = np.arange(0.6, 1 + 0.1, 0.4)
rescale_y = rescale_x

In [7]:
from itertools import product

In [8]:
combinations = list(product(angles, shift_x, shift_y, fliplr, rescale_x, rescale_y))
len(combinations)

1000

In [9]:
def modify_photo(img, angle, shift_x, shift_y, fliplr, rescale_x, rescale_y):
    # rescale y
    rescaled_y = transform.rescale(img, (rescale_y, 1))
    background = np.ones((int(np.ceil(150 * (1 - rescale_y) * 0.5)), 150)) * np.mean(im)
    img = np.concatenate((background, rescaled_y, background), axis=0)
    
    # rescale x
    rescaled_x = transform.rescale(img, (1, rescale_x))
    background = np.ones((150, int(np.ceil(150 * (1 - rescale_x) * 0.5)))) * np.mean(im)
    img = np.concatenate((background, rescaled_x, background), axis=1)
    
    # rotate
    img = transform.rotate(img, 5, cval=np.mean(img))
    
    # shift
    img = shift(img, (shift_x, shift_y), cval=np.mean(im))
    
    # flip
    if fliplr:
        img = np.fliplr(img)
    
    fname = f"{angle}-{shift_x}-{shift_y}-{fliplr}-{rescale_x}-{rescale_y}"
    # plt.imsave(f"data/syntetic/{fname}.jpg", img)
    return img

In [10]:
prisms = [
    f"data/originals/PRISM{i}.jpg"
    for i in range(1, 10)
]

In [11]:
prisms_images = []

In [12]:
for prism in prisms:
    im = Image.open(prism)
    im = im.convert('L')
    im = np.asarray(im)
    im = transform.resize(im, (150, 150))
    for i in tqdm_notebook(range(len(combinations))):
        prisms_images.append(
            modify_photo(im, *combinations[i])
        )

/home/bartek/.pyenv/versions/3.6.1/envs/woarld/lib/python3.6/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


In [13]:
pyramids = [
    f"data/originals/PYRAMIDE{i}.jpg"
    for i in range(1, 11)
]

In [14]:
pyramids_images = []

In [15]:
for pyramid in pyramids:
    im = Image.open(pyramid)
    im = im.convert('L')
    im = np.asarray(im)
    im = transform.resize(im, (150, 150))
    for i in tqdm_notebook(range(len(combinations))):
        pyramids_images.append(
            modify_photo(im, *combinations[i])
        )

### Preprocess samples

In [16]:
def preprocess(img):
    threshold = filters.threshold_otsu(img)
    img = img < threshold
    img = morphology.binary_dilation(img)
    img = transform.resize(img, (50, 50))
    return img

In [17]:
preprocessed_prisms = [preprocess(prism) for prism in prisms_images]

In [18]:
preprocessed_pyramids = [preprocess(pyramid) for pyramid in pyramids_images]

In [30]:
X_prisms = np.array([prism.flatten() for prism in preprocessed_prisms])
x, y = X_prisms.shape
Xy_prisms = np.zeros((x, y+1))
Xy_prisms[:, :-1] = X_prisms

In [31]:
X_pyramids = np.array([pyramid.flatten() for pyramid in preprocessed_pyramids])
x, y = X_pyramids.shape
Xy_pyramids = np.ones((x, y+1))
Xy_pyramids[:, :-1] = X_pyramids

### Classification

In [44]:
Xy = np.concatenate((Xy_prisms, Xy_pyramids))
np.random.shuffle(Xy)
X = Xy[:, :-1]
y = Xy[:, -1]

In [51]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3)

In [49]:
knn = neighbors.KNeighborsClassifier(5)

In [53]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [55]:
knn.score(X_test, y_test)

0.9673684210526315